(api/01-fastapi)=
# API development 1

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/ok-transformer/blob/master/docs/nb/api/01-fastapi.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/ok-transformer?style=social)](https://github.com/particle1331/ok-transformer)

---

**Readings:**  {cite}`timviera`

## Introduction

In this chapter, we’ll cover the following main topics:
- Creating the first endpoint and running it locally
- Handling request parameters
- Customizing the response
- Structuring a bigger project with multiple routers

## Our first endpoint

The following creates a GET endpoint in the root path `/` using https://github.com/tiangolo/fastapi. 
The `hello_world` coroutine (see appendix) contains our route logic. This also called a **path operation** function. 
The return value is a dictionary that is automatically handled by FastAPI as a JSON response. Note that we use a decorator 
to specify the [HTTP method](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods) on the given route.

In [1]:
%%writefile ./01/main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def hello_world():
    return {"hello": "world"}

Overwriting ./01/main.py


Running in development mode allowing **reloads** of the server each time a code chage is made:

In [2]:
import time
import subprocess

cmd = "uvicorn 01.main:app --host 0.0.0.0 --port 8000 --reload"
process = subprocess.Popen(cmd.split())
print("pid:", process.pid)
time.sleep(1.0)

pid: 73779


INFO:     Will watch for changes in these directories: ['/Users/particle1331/code/ok-transformer/docs/nb/apis']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [73779] using WatchFiles


This means that the root path is in `http://0.0.0.0:8000/`. Here we use https://github.com/httpie/cli to simply our requests. The following command defaults to a GET request on localhost. Note that the output is a JSON object:

In [3]:
!http :8000/

INFO:     Started server process [73804]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


INFO:     127.0.0.1:52742 - "GET / HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 17
content-type: application/json
date: Wed, 19 Jun 2024 21:07:40 GMT
server: uvicorn

{
    "hello": "world"
}




**Remark.** An endpoint can have no return value (i.e. returns `None`). Here the response is an empty JSON (`null`).

FastAPI automatically provides interactive documentation:

```{figure} ../../img/apis/01-swagger.png
---
name: 01-swagger
width: 800px
---
Automatic documentation in `http://0.0.0.0:8000/docs`
```

## Request parameters

The main goal of an API is to provide a structured way to interact
with data. As such, it is crucial for the **client** to send information based on the 
response they need, such as path parameters, query parameters, body payloads, headers, and so on.
From the perspective of the **server**, request data should be validated based on the endpoint
logic.

### Path parameters

Resources can be accessed dynamically from paths using **path parameters**: 

In [4]:
%%writefile ./01/main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/users/{id}")
async def get_user(id: int):
    return {"id": id}

Overwriting ./01/main.py


**Remark.** Our paths always start with an `/` and does not end on a `/`.

In [5]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [73804]
INFO:     Started server process [73877]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Note that type hint is crucial. This can be seen in the JSON response where `id` is cast as int:

In [6]:
!http :8000/users/3

INFO:     127.0.0.1:52746 - "GET /users/3 HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 8
content-type: application/json
date: Wed, 19 Jun 2024 21:07:41 GMT
server: uvicorn

{
    "id": 3
}




If the value cannot be cast to int, we get an error:

In [7]:
!http :8000/users/abc

INFO:     127.0.0.1:52750 - "GET /users/abc HTTP/1.1" 422 Unprocessable Entity
HTTP/1.1 422 Unprocessable Entity
content-length: 147
content-type: application/json
date: Wed, 19 Jun 2024 21:07:42 GMT
server: uvicorn

{
    "detail": [
        {
            "input": "abc",
            "loc": [
                "path",
                "id"
            ],
            "msg": "Input should be a valid integer, unable to parse string as an integer",
            "type": "int_parsing"
        }
    ]
}




Note that we can apply validation on the path parameters. Multiple path parameters are also allowed:

In [8]:
%%writefile ./01/main.py
from enum import Enum
from typing import Annotated
from fastapi import FastAPI, Path

class UserType(str, Enum):
    STANDARD = "standard"
    ADMIN = "admin"


app = FastAPI()

@app.get("/users/{type}/{id}")
async def get_user(type: UserType, id: Annotated[int, Path(ge=0)]):
    return {"type": type, "id": id}

Overwriting ./01/main.py


**Remark.** Note the use of **type annotation** (Python 3.9+). 
This provides FastAPI with additional metadata about how we want the application should behave. 
The first type passed is the *actual type* of the parameter. Here we add the `Path` to validate
the path parameter to be at least 0. Finally, a parameter that has no default value is **required**.

In [9]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [73877]
INFO:     Started server process [73931]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Using enum constrains the user type to either `"standard"` or `"admin"`:

In [10]:
!http :8000/users/test/0

INFO:     127.0.0.1:52754 - "GET /users/test/0 HTTP/1.1" 422 Unprocessable Entity
HTTP/1.1 422 Unprocessable Entity
content-length: 154
content-type: application/json
date: Wed, 19 Jun 2024 21:07:43 GMT
server: uvicorn

{
    "detail": [
        {
            "ctx": {
                "expected": "'standard' or 'admin'"
            },
            "input": "test",
            "loc": [
                "path",
                "type"
            ],
            "msg": "Input should be 'standard' or 'admin'",
            "type": "enum"
        }
    ]
}




Using a valid `type` value:

In [11]:
!http :8000/users/standard/3

INFO:     127.0.0.1:52758 - "GET /users/standard/3 HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 26
content-type: application/json
date: Wed, 19 Jun 2024 21:07:44 GMT
server: uvicorn

{
    "id": 3,
    "type": "standard"
}




In [12]:
!http :8000/users/admin/0

INFO:     127.0.0.1:52762 - "GET /users/admin/0 HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 23
content-type: application/json
date: Wed, 19 Jun 2024 21:07:45 GMT
server: uvicorn

{
    "id": 0,
    "type": "admin"
}




Trying an invalid id:

In [13]:
!http :8000/users/admin/-1

INFO:     127.0.0.1:52767 - "GET /users/admin/-1 HTTP/1.1" 422 Unprocessable Entity
HTTP/1.1 422 Unprocessable Entity
content-length: 141
content-type: application/json
date: Wed, 19 Jun 2024 21:07:45 GMT
server: uvicorn

{
    "detail": [
        {
            "ctx": {
                "ge": 0
            },
            "input": "-1",
            "loc": [
                "path",
                "id"
            ],
            "msg": "Input should be greater than or equal to 0",
            "type": "greater_than_equal"
        }
    ]
}




**Remark.** For a string path parameter, validation such as the following exists (regex also possible):

```
username: Annotated[str, Path(min_length=4, max_length=16)]
```

### Query parameters

Query parameters are another common way to add dynamic parameters to a URL. These can be found at the end of the URL in the following form: `?param1=foo&param2=bar`. They are commonly used on read endpoints to apply pagination, a filter, a sorting order, or selecting fields. The following implements two optional query parameters for pagination:

In [14]:
%%writefile ./01/main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/users")
async def get_users(offset: int = 0, limit: int = 100):
    return {"offset": offset, "limit": limit}

Overwriting ./01/main.py


In [15]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [73931]
INFO:     Started server process [74028]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [16]:
!http "http://localhost:8000/users?offset=5&limit=10"

INFO:     127.0.0.1:52771 - "GET /users?offset=5&limit=10 HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 23
content-type: application/json
date: Wed, 19 Jun 2024 21:07:47 GMT
server: uvicorn

{
    "limit": 10,
    "offset": 5
}




**Remark.** HTTPie uses `param==value` syntax for query parameters:

In [17]:
!http :8000/users offset==5 limit==10

INFO:     127.0.0.1:52775 - "GET /users?offset=5&limit=10 HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 23
content-type: application/json
date: Wed, 19 Jun 2024 21:07:48 GMT
server: uvicorn

{
    "limit": 10,
    "offset": 5
}




The following example shows selecting fields using query parameters:

In [18]:
%%writefile ./01/main.py
from typing import Annotated
from fastapi import FastAPI, Query

app = FastAPI()

# Sample data
items = [
    {"id": 1, "name": "Item 1", "description": "Description 1", "price": 10.0},
    {"id": 2, "name": "Item 2", "description": "Description 2", "price": 20.0},
]

allowed_fields = {"id", "name", "description", "price"}


@app.get("/items")
def read_items(fields: Annotated[list[str], Query()] = list(allowed_fields)):
    result = []
    for item in items:
        filtered_item = {field: item[field] for field in fields if field in item}
        result.append(filtered_item)
        
    return result

Overwriting ./01/main.py


The allowed fields have a default value that are conveniently listed in the docs:

```{figure} ../../img/apis/01-swagger-list-string.png
---
name: 01-swagger-list-string
width: 800px
---
```

In [19]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74028]
INFO:     Started server process [74085]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Default behavior:

In [20]:
!http :8000/items

INFO:     127.0.0.1:52779 - "GET /items HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 137
content-type: application/json
date: Wed, 19 Jun 2024 21:07:50 GMT
server: uvicorn

[
    {
        "description": "Description 1",
        "id": 1,
        "name": "Item 1",
        "price": 10.0
    },
    {
        "description": "Description 2",
        "id": 2,
        "name": "Item 2",
        "price": 20.0
    }
]




Filtering:

In [21]:
!http :8000/items fields==id fields==name

INFO:     127.0.0.1:52783 - "GET /items?fields=id&fields=name HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 51
content-type: application/json
date: Wed, 19 Jun 2024 21:07:51 GMT
server: uvicorn

[
    {
        "id": 1,
        "name": "Item 1"
    },
    {
        "id": 2,
        "name": "Item 2"
    }
]




**Remark.** Validation functionality in `Path` are also available for `Query`. In the above code, we can also raise `HTTPException` (imported from the `fastapi` library) if the given field is not in the allowed fields:

```python
allowed_fields = {"id", "name", "description", "price"}

@app.get("/items")
def read_items(fields: Annotated[list[str], Query()] = list(allowed_fields)):
    for field in fields:
        if field not in allowed_fields:
            raise HTTPException(status_code=400, detail=f"Field '{field}' is not valid")
    
    ...
```

Note that `fields` will never be empty, since if do not provide a field query, then it defaults to the allowed fields.

### Request body

The body is the part of the HTTP request that contains raw data representing documents, files, or
form submissions. In a REST API, it is usually encoded in JSON and used to create structured objects
in a database. Note that we use a POST request to send data in a request body:

In [22]:
%%writefile ./01/main.py
from typing import Annotated
from fastapi import FastAPI, Body

app = FastAPI()

@app.post("/users")
async def create_user(name: Annotated[str, Body()], age: Annotated[int, Body(ge=0)]):
    return {"name": name, "age": age}

Overwriting ./01/main.py


In [23]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74085]


We have to specify POST since HTTPie defaults to GET. The syntax for body fields is `field=value`:

In [24]:
!http -v POST :8000/users name=John age=0

INFO:     Started server process [74140]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


POST /users HTTP/1.1
Accept: application/json, */*;q=0.5
Accept-Encoding: gzip, deflate
Connection: keep-alive
Content-Length: 28
Content-Type: application/json
Host: localhost:8000
User-Agent: HTTPie/3.2.2

{
    "age": "0",
    "name": "John"
}


INFO:     127.0.0.1:52788 - "POST /users HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 23
content-type: application/json
date: Wed, 19 Jun 2024 21:07:53 GMT
server: uvicorn

{
    "age": 0,
    "name": "John"
}




The `-v` flag for HTTPie allows us to see the JSON payload we sent. Both fields are strings, but age is later cast as int once processed. Since each field is required, we get a 422 status error response if we send a request with a missing field. Note that request body fields also have validation functionality same as query and path parameters above.

```{margin}
https://github.com/pydantic/pydantic
```

**Schemas.** In practice, we will reuse the structure of the data between multiple endpoints (e.g. a document, or a user). Hence, it would be nice to define a **schema** for the request body. This makes it easy to read and refactor code. FastAPI uses **Pydantic models** to accomplish this. Pydantic is a library for data validation and is based on classes and type hints which makes it ideal for this use case. 

Modifying the above code:

In [25]:
%%writefile ./01/main.py
from fastapi import FastAPI
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str
    age: int = Field(ge=0)

app = FastAPI()

@app.post("/users")
async def create_user(user: User):
    return user

Overwriting ./01/main.py


In [26]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74140]
INFO:     Started server process [74185]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Note that Pydantic model output is automatically converted to JSON:

In [27]:
!http POST :8000/users name=John age=1

INFO:     127.0.0.1:52792 - "POST /users HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 23
content-type: application/json
date: Wed, 19 Jun 2024 21:07:54 GMT
server: uvicorn

{
    "age": 1,
    "name": "John"
}




Testing **field validation**:

In [28]:
!http POST :8000/users name=John age=-1

INFO:     127.0.0.1:52796 - "POST /users HTTP/1.1" 422 Unprocessable Entity
HTTP/1.1 422 Unprocessable Entity
content-length: 142
content-type: application/json
date: Wed, 19 Jun 2024 21:07:55 GMT
server: uvicorn

{
    "detail": [
        {
            "ctx": {
                "ge": 0
            },
            "input": "-1",
            "loc": [
                "body",
                "age"
            ],
            "msg": "Input should be greater than or equal to 0",
            "type": "greater_than_equal"
        }
    ]
}




**Remark.** 
Each individual attribute can be accessed by using the dot notation, such as `user.name`.
Field filtering can be done with `item.model_dump(include=set(fields))` where `item` is a Pydantic model. 
We can use multiple models as well as a single body parameter for a property that is not part of any model:

In [29]:
%%writefile ./01/main.py
from typing import Annotated
from fastapi import FastAPI, Body
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str
    age: int = Field(ge=0)

app = FastAPI()

@app.post("/users")
async def create_user(user: User, priority: Annotated[int, Body()]):
    return {"user": user, "priority": priority}

Overwriting ./01/main.py


In [30]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74185]
INFO:     Started server process [74239]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Here we have to pipe the more complex JSON request:

In [31]:
!http POST :8000/users <<< '{"user": {"name": "John", "age": 3}, "priority": 3}'


INFO:     127.0.0.1:52800 - "POST /users HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 45
content-type: application/json
date: Wed, 19 Jun 2024 21:07:57 GMT
server: uvicorn

{
    "priority": 3,
    "user": {
        "age": 3,
        "name": "John"
    }
}




**Remark.** The above examples show that schemas ensures that data is validated as well as properly serialized between different formats. Schemas are also used when automatically generating API documentation.

### File uploads

FastAPI provides a parameter function `File` that enables file uploads. This is a common requirement for web applications (e.g. image, audio, or document upload). The approach is as usual: we can use `file: Annotated[bytes, File()]` where `File` is the parameter function provided by FastAPI. This stores the data in memory, and so may not be ideal for large files. 

FastAPI provides an `UploadFile` class will store the data in memory up to a certain threshold, then will automatically store it on disk in a **temporary location**. This allows you to accept much larger files without running out of memory. Furthermore, the object instance exposes useful metadata (e.g. `file.content_type`) and a file-like interface. This means
that it can be manipulated as a regular file in Python (e.g. `file.read`, `file.write`) and feed it to functions that expect a file.

In [32]:
%%writefile ./01/main.py
import numpy as np
from PIL import Image
from fastapi import FastAPI, UploadFile

app = FastAPI()

@app.post("/files")
async def upload_file(file: UploadFile):
    img = Image.open(file.file)
    return {
        "filename": file.filename,
        "content_type": file.content_type,
        "size": file.size,
        "dims": np.array(img).shape, 
    }

Overwriting ./01/main.py


In [33]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74239]
INFO:     Started server process [74281]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


HTTPie upload syntax. Note curl is a bit different with `file=@` instead of `file@`:

In [34]:
!http --form POST :8000/files file@./assets/tom.jpeg

INFO:     127.0.0.1:52804 - "POST /files HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 84
content-type: application/json
date: Wed, 19 Jun 2024 21:07:58 GMT
server: uvicorn

{
    "content_type": "image/jpeg",
    "dims": [
        923,
        923,
        3
    ],
    "filename": "tom.jpeg",
    "size": 130055
}




**Remark.** The content type is particularly useful. The upload can be rejected if the server receives an unexpected filetype. Multiple uploads is also possible:

```python
@app.post("/files")
async def upload_files(files: list[UploadFile]):
    return [
        {
            "filename": file.filename,
            "content_type": file.content_type,
            "size": file.size,
            "dims": np.array(Image.open(file.file)).shape, 
        }
        for file in files
    ]
```

Multiple upload request using HTTPie:

```
http --form POST :8000/files files@xgboost-parallel.png files@xgboost-contour.png files@workflow.png
```

Equivalent execution in the docs UI:

```{figure} ../../img/apis/01-swagger-file-uploads.png
---
name: 01-swagger-file-uploads
width: 800px
---
Multiple file uploads in the docs UI.
```


## Customizing the response

In the previous examples, our endpoints simply returned a dictionary or Pydantic object which is enough for FastAPI to return a JSON response. Most of the time, we want to customize the response further: add status code, raising errors, and setting cookies. We will consider multiple ways of doing this with increasing customization.

### Decorator parameters

The following modifies the status code from `200 OK` to `201 Created` via the path decorator. Note that the specified status code only applies if no errors were encountered. Other status codes are `204 No content` for a delete endpoint (i.e. `@app.delete`)

In [35]:
%%writefile ./01/main.py
from pydantic import BaseModel
from fastapi import FastAPI, status

class Post(BaseModel):
    title: str

app = FastAPI()

@app.post("/posts", status_code=status.HTTP_201_CREATED)
async def create_post(post: Post):
    return post

Overwriting ./01/main.py


In [36]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74281]
INFO:     Started server process [74321]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Note that response status is `201 Created`:

In [37]:
!http POST :8000/posts title="Post"

INFO:     127.0.0.1:52808 - "POST /posts HTTP/1.1" 201 Created
HTTP/1.1 201 Created
content-length: 16
content-type: application/json
date: Wed, 19 Jun 2024 21:08:00 GMT
server: uvicorn

{
    "title": "Post"
}




### Response model

The main use case in FastAPI is to directly return a Pydantic model that is automatically serialized to JSON. However, it is often the case that there are some differences between
the input data, the data you store in your database, and the data you want to show to the end user. For example, some fields are private or only used in internal logic, or only used temporarily.

## Appendix: Other parameters

### Form data

HTML forms (`<form></form>`) sends the data to a server normally using a different encoding for that data (i.e. different from JSON). FastAPI will make sure to read that data from the right place instead of JSON.

In [38]:
%%writefile ./01/main.py
from typing import Annotated
from fastapi import FastAPI, Form

app = FastAPI()

@app.post("/users")
async def create_user(name: Annotated[str, Form()], age: Annotated[int, Form()]):
    return {"name": name, "age": age}

Overwriting ./01/main.py


In [39]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74321]
INFO:     Started server process [74363]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Request syntax:

In [40]:
!http -v --form :8000/users name=John age=30

POST /users HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Content-Length: 16
Content-Type: application/x-www-form-urlencoded; charset=utf-8
Host: localhost:8000
User-Agent: HTTPie/3.2.2

INFO:     127.0.0.1:52812 - "POST /users HTTP/1.1" 200 OK
name=John&age=30


HTTP/1.1 200 OK
content-length: 24
content-type: application/json
date: Wed, 19 Jun 2024 21:08:02 GMT
server: uvicorn

{
    "age": 30,
    "name": "John"
}




### Headers and cookies

FastAPI automatically retrieves **headers** by converting to lowercase and replacing `-` with `_`. 
FastAPI also provides a way to get **cookies** which are a special type of header. These will be helpful in implementing common authentication schemes.

In [41]:
%%writefile ./01/main.py
from typing import Annotated
from fastapi import FastAPI, Cookie, Header

app = FastAPI()

@app.get("/cookie")
async def get_cookie(hello: Annotated[str | None, Cookie()] = None):
    return {"hello": hello}

@app.get("/header")
async def get_header(user_agent: Annotated[str, Header()], connection: Annotated[str, Header()]):
    return {"user_agent": user_agent, "connection": connection}

Overwriting ./01/main.py


In [42]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74363]
INFO:     Started server process [74404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Sample requests:

In [43]:
!http :8000/cookie

INFO:     127.0.0.1:52816 - "GET /cookie HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 14
content-type: application/json
date: Wed, 19 Jun 2024 21:08:04 GMT
server: uvicorn

{
    "hello": null
}




In [44]:
!http -v :8000/header

INFO:     127.0.0.1:52820 - "GET /header HTTP/1.1" 200 OK
GET /header HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: localhost:8000
User-Agent: HTTPie/3.2.2



HTTP/1.1 200 OK
content-length: 55
content-type: application/json
date: Wed, 19 Jun 2024 21:08:05 GMT
server: uvicorn

{
    "connection": "keep-alive",
    "user_agent": "HTTPie/3.2.2"
}




### Request object

The **request object** is used to access raw data associated
with a request:

In [45]:
%%writefile ./01/main.py
from fastapi import FastAPI, Request

app = FastAPI()

@app.post("/")
async def get_request_object(request: Request):
    return {
        "path": request.url.path,
        "hostname": request.base_url.hostname,
        "port": request.base_url.port,
    }

Overwriting ./01/main.py


In [46]:
time.sleep(1)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74404]
INFO:     Started server process [74455]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


As usual, we simply had to declare an argument hinted with
the `Request` class.

In [47]:
!http POST :8000/

INFO:     127.0.0.1:52824 - "POST / HTTP/1.1" 200 OK
HTTP/1.1 200 OK
content-length: 47
content-type: application/json
date: Wed, 19 Jun 2024 21:08:06 GMT
server: uvicorn

{
    "hostname": "localhost",
    "path": "/",
    "port": 8000
}




## Appendix: Asynchronous I/O 

I/O operations such as reading from disk or network requests are million times slower than reading
from RAM or processing instructions.
The **asynchronous paradigm** is a way to make I/O operations non-blocking and allow the program to perform other
tasks while the slow read or write operation is ongoing.
For web servers, waiting on I/O operations could be an opportunity to perform other tasks.
This has been achieved through the concept of an **event loop**. The event loop orchestrates
all the asynchronous tasks sent to it. When data is available or when the write operation
is done for one of those tasks, it will ping the main program so that it can perform the next operations.

In [48]:
import asyncio

async def main(delay: int = 1):
    print("Hello...")
    if delay > 0:
        await asyncio.sleep(delay)
    print("... World!")

await main()

Hello...


... World!


This runs for about 1 sec. The `async` keyword indicates that `main` is an asynchronous function or **coroutine**. This allows us to use `await` inside it. Note that `asyncio.sleep` is also a coroutine that is the asynchronous version of `time.sleep`. Prefixing with `await` sends it to the event loop, but also means that we want to go back to it once it completes, and continue the next operations (i.e. the next print statement). 
Hence, such operations are said to be **non-blocking**.
This allows the event loop to process other tasks in the mean time:

In [49]:
async def other():
    print("<☺>")

await asyncio.gather(main(), other())

Hello...
<☺>
... World!


[None, None]

Note that `gather` collects the outputs of each coroutine in a list which we await. The coroutines are executed **concurrently**. If `main` does not have the I/O operation, then it continues to block the event loop until it has completed processing. Regular operations such as computations are blocking and will block the event loop. Usually, this is not a problem since those
operations are fast (as is the case here):

In [50]:
await asyncio.gather(main(0), other())

Hello...
... World!
<☺>


[None, None]

The only operations that are non-blocking are I/O operations designed
to work asynchronously. This is different from **multiprocessing** where operations are executed on
child processes, so blocking the main process clearly does not happen. Because of this, we will have to 
be careful when choosing a third-party library for interacting with databases, APIs, and so on, if they are 
not designed to work asynchronously.

---

■